##  Neo4j_Analysis_Tmp



1. 找到连接机场最多的机场（度中心性分析）

商业意义：找出最重要的Hub机场，决定航班布局、投资重点。

MATCH (a:Airport)
RETURN a.name AS airport_name,
       size((a)-[:FLIES_TO]->()) + size(()-[:FLIES_TO]->(a)) AS total_connections
ORDER BY total_connections DESC
LIMIT 10



⸻

2. 识别哪些航班是被多个航空公司运营的（共享航班热点）

商业意义：找出竞争最激烈的航班，调整价格或增开班次。

MATCH (al:Airline)-[:OPERATES]->(f:Flight)
WITH f.flight_id AS flight_id, COUNT(DISTINCT al) AS airline_count
WHERE airline_count > 1
RETURN flight_id, airline_count
ORDER BY airline_count DESC
LIMIT 10



⸻

3. 查找最常飞行的乘客（飞行次数最多的乘客）

商业意义：识别高价值客户（Frequent Flyers），可以做会员营销。

MATCH (p:Passenger)-[:BOOKED]->(f:Flight)
RETURN p.first_name + ' ' + p.last_name AS passenger_name,
       COUNT(f) AS flight_count
ORDER BY flight_count DESC
LIMIT 10



⸻

4. 查询指定机场到其他机场最短路径（最少中转机场）

商业意义：优化航线设计，减少中转，提高乘客体验。

比如从 “Los Angeles International Airport” 出发到 “John F Kennedy International Airport”：

MATCH (start:Airport {name: "Los Angeles International Airport"}),
      (end:Airport {name: "John F Kennedy International Airport"}),
      p = shortestPath((start)-[:FLIES_TO*..10]->(end))
RETURN p



⸻

5. 找出最常用的飞机型号（Flight -> Aircraft）

商业意义：看哪些机型需要优先维护、更换、追加采购。

MATCH (f:Flight)-[:USES]->(ac:Aircraft)
RETURN ac.model AS aircraft_model,
       COUNT(f) AS flight_count
ORDER BY flight_count DESC
LIMIT 10



⸻

6. 查找从某个国家起飞最多的机场（基于机场 country属性）

商业意义：做区域市场开发，比如美国、中国、欧洲重点国家。

MATCH (a:Airport)-[:FLIES_TO]->()
WHERE a.country = "United States"
RETURN a.name AS airport_name,
       COUNT(*) AS outbound_routes
ORDER BY outbound_routes DESC
LIMIT 10



⸻

7. 查询延误航班最多的航线（基于Flight的status）

商业意义：找出经常延误的航线，优化时刻表或者加强运营协调。

MATCH (a1:Airport)-[:FLIES_TO]->(a2:Airport),
      (f:Flight)
WHERE f.status = "Delayed"
AND f.flight_id IS NOT NULL
RETURN a1.name AS source_airport,
       a2.name AS destination_airport,
       COUNT(f) AS delay_count
ORDER BY delay_count DESC
LIMIT 10





In [11]:
from neo4j import GraphDatabase

# Neo4j 数据库信息
NEO4J_URI = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""

# 创建 Neo4j 驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
# 验证连接
with driver.session() as session:
    session.run("RETURN 1")
    print("连接成功！")

# 用来跑 query 的 function
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# ⭐查询数据, 引号""里面换成你的 query ⭐
query22 = """
MATCH (a:Airport)
RETURN a.name AS airport_name,
       COUNT { (a)-[:FLIES_TO]->() } + COUNT { ()-[:FLIES_TO]->(a) } AS total_connections
ORDER BY total_connections DESC
LIMIT 10
"""
all_nodes = run_query(query22)
print(all_nodes)

连接成功！
[{'airport_name': 'Ericachester International Airport', 'total_connections': 7}, {'airport_name': 'Lake Thomasmouth International Airport', 'total_connections': 6}, {'airport_name': 'Port Laura International Airport', 'total_connections': 4}, {'airport_name': 'West Anne International Airport', 'total_connections': 3}, {'airport_name': 'New Carol International Airport', 'total_connections': 3}, {'airport_name': 'Carolinemouth International Airport', 'total_connections': 3}, {'airport_name': 'Josephstad International Airport', 'total_connections': 3}, {'airport_name': 'West Charles International Airport', 'total_connections': 3}, {'airport_name': 'Castroland International Airport', 'total_connections': 2}, {'airport_name': 'New Brooke International Airport', 'total_connections': 2}]


In [10]:
from pyvis.network import Network

# 创建 pyvis 的 Network
net = Network(notebook=True, directed=True)

# ⭐这里换成你的 query ⭐
query11 = """
MATCH (a:Airport)
RETURN a.name AS airport_name,
       COUNT { (a)-[:FLIES_TO]->() } + COUNT { ()-[:FLIES_TO]->(a) } AS total_connections
ORDER BY total_connections DESC
LIMIT 10
"""

data = run_query(query11)

# 加入节点和边
for record in data:
    n = record['n']
    m = record['m']
    r = record['r']

    # 从字典中取 name，如果没有就用str(n)做id
    n_id = n.get('name', str(n))
    m_id = m.get('name', str(m))

    # 给节点加label（可选）
    n_label = ", ".join(n.keys()) if n else "Node"
    m_label = ", ".join(m.keys()) if m else "Node"

    # 添加节点（避免重复）
    net.add_node(n_id, label=n_id, title=str(n))
    net.add_node(m_id, label=m_id, title=str(m))

    # 取关系类型
    relation_type = r.get('type', 'RELATES_TO') if isinstance(r, dict) else 'RELATES_TO'

    # 添加边
    net.add_edge(n_id, m_id, label=relation_type)

# 在Jupyter Notebook里显示
net.show("graph.html")

KeyError: 'n'

好嘞！
既然你现在 Neo4j 的数据结构已经搭好（Airport, Airline, Flight, Passenger, Aircraft，还有各种关系），
那我就帮你想一套：

针对图结构特性设计的问题，不是普通SQL能做的，要用图查询的方式去挖掘隐藏信息。

要求符合你之前说的：
	•	有实际商业价值
	•	难度稍微高一点（不是简单MATCH）
	•	每个问题配上 Cypher 查询

⸻

🧠 Neo4j 分析问题 + Cypher 查询（商业版）

⸻

1. 找到连接机场最多的机场（度中心性分析）

商业意义：找出最重要的Hub机场，决定航班布局、投资重点。

MATCH (a:Airport)
RETURN a.name AS airport_name,
       size((a)-[:FLIES_TO]->()) + size(()-[:FLIES_TO]->(a)) AS total_connections
ORDER BY total_connections DESC
LIMIT 10



⸻

2. 识别哪些航班是被多个航空公司运营的（共享航班热点）

商业意义：找出竞争最激烈的航班，调整价格或增开班次。

MATCH (al:Airline)-[:OPERATES]->(f:Flight)
WITH f.flight_id AS flight_id, COUNT(DISTINCT al) AS airline_count
WHERE airline_count > 1
RETURN flight_id, airline_count
ORDER BY airline_count DESC
LIMIT 10



⸻

3. 查找最常飞行的乘客（飞行次数最多的乘客）

商业意义：识别高价值客户（Frequent Flyers），可以做会员营销。

MATCH (p:Passenger)-[:BOOKED]->(f:Flight)
RETURN p.first_name + ' ' + p.last_name AS passenger_name,
       COUNT(f) AS flight_count
ORDER BY flight_count DESC
LIMIT 10



⸻

4. 查询指定机场到其他机场最短路径（最少中转机场）

商业意义：优化航线设计，减少中转，提高乘客体验。

比如从 “Los Angeles International Airport” 出发到 “John F Kennedy International Airport”：

MATCH (start:Airport {name: "Los Angeles International Airport"}),
      (end:Airport {name: "John F Kennedy International Airport"}),
      p = shortestPath((start)-[:FLIES_TO*..10]->(end))
RETURN p



⸻

5. 找出最常用的飞机型号（Flight -> Aircraft）

商业意义：看哪些机型需要优先维护、更换、追加采购。

MATCH (f:Flight)-[:USES]->(ac:Aircraft)
RETURN ac.model AS aircraft_model,
       COUNT(f) AS flight_count
ORDER BY flight_count DESC
LIMIT 10



⸻

6. 查找从某个国家起飞最多的机场（基于机场 country属性）

商业意义：做区域市场开发，比如美国、中国、欧洲重点国家。

MATCH (a:Airport)-[:FLIES_TO]->()
WHERE a.country = "United States"
RETURN a.name AS airport_name,
       COUNT(*) AS outbound_routes
ORDER BY outbound_routes DESC
LIMIT 10



⸻

7. 查询延误航班最多的航线（基于Flight的status）

商业意义：找出经常延误的航线，优化时刻表或者加强运营协调。

MATCH (a1:Airport)-[:FLIES_TO]->(a2:Airport),
      (f:Flight)
WHERE f.status = "Delayed"
AND f.flight_id IS NOT NULL
RETURN a1.name AS source_airport,
       a2.name AS destination_airport,
       COUNT(f) AS delay_count
ORDER BY delay_count DESC
LIMIT 10



⸻

✨ 小总结表

问题	商业价值	查询难度
找连接最多机场	Hub识别/航线布局	中
多航空公司运营的航班	市场竞争分析	中
最常飞的乘客	客户关系管理	简单
最短路径查询	航线优化	中偏高
最常用飞机型号	资产管理	简单
单国最多起飞机场	区域市场分析	简单
最多延误的航线	运营优化	中



⸻

🚀 如果想更难一点（比如数据科学级的图分析）

可以考虑：
	•	PageRank 排序机场重要性（模拟Google的搜索权重）
	•	Community Detection（机场之间形成的区域性子网络）
	•	路径预测（基于图卷积神经网络GCN预测潜在航线）
	•	乘客社交网络分析（同航班乘客共飞次数，构建社交图）

这些就超难了，适合研究型项目，但我也可以带你做！

⸻

要不要我顺便也打包一版：
	•	直接一键跑 PageRank
	•	跑最短路径 shortestPath()
	•	跑简单社区发现 (node clustering)

给你弄成可直接复制到 Neo4j Browser 的指令？🚀
要的话告诉我，继续打包给你！🎯
继续吗？🌟